In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader
import seaborn as sns
import matplotlib.pyplot as plt
from torchvision import datasets
import numpy as np
import pandas as pd
from adjustText import adjust_text
import random
import os
from model_structure import get_preprocessing_transforms,BCNN, train_model, evaluate_model

In [7]:
INPUT_SIZE = 224
BATCH_SIZE = 32
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
# Get transforms
train_transform, val_transform = get_preprocessing_transforms(INPUT_SIZE)

test_data_1_dir = 'data4model_1/test/'
test_data_2_dir = 'data4model_2/test/'
train_data_1_dir = 'data4model_1/train/'
train_data_2_dir = 'data4model_2/train/'

# Load data set
dataset_test_1 = datasets.ImageFolder(test_data_1_dir,transform=val_transform)
dataset_train_1 = datasets.ImageFolder(train_data_1_dir,transform=val_transform)
dataset_test_2 = datasets.ImageFolder(test_data_2_dir,transform=val_transform)
dataset_train_2 = datasets.ImageFolder(train_data_2_dir,transform=val_transform)
additional_set = datasets.ImageFolder('data4model_1/for_extra_test/',transform=val_transform)



additional_loader = DataLoader(additional_set, shuffle=False, batch_size=BATCH_SIZE)
test_loader_1 = DataLoader(dataset_test_1, shuffle=False, batch_size=BATCH_SIZE)
train_loader_1 = DataLoader(dataset_train_1, shuffle=False, batch_size=BATCH_SIZE)
test_loader_2 = DataLoader(dataset_test_2, shuffle=False, batch_size=BATCH_SIZE)
train_loader_2 = DataLoader(dataset_train_2, shuffle=False, batch_size=BATCH_SIZE)

dataSets =  {"Model_1:Train": train_loader_1,
        "Model_1:Test": test_loader_1,
        "Model_2:Train": train_loader_2,
        "Model_2:Test": test_loader_2,
        "Model_1:additional_set": additional_loader
        }

In [9]:
model_1_dir = 'best_model_1.pth'
model_2_dir = 'best_model_2.pth'

In [10]:
# Load the model
model_1 = BCNN().to(DEVICE)
weights = torch.load(model_1_dir, map_location=torch.device(DEVICE))
model_1.load_state_dict(weights)
model_1.eval()

model_2 = BCNN().to(DEVICE)
weights = torch.load(model_2_dir, map_location=torch.device(DEVICE))
model_2.load_state_dict(weights)
model_2.eval()

C:\Users\hskay\AppData\Local\Temp\ipykernel_18232\1438434258.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(model_1_dir, map_location=torch.device(

BCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout2d(p=0.25, inplace=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout2d(p=0.25, inplace=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Dropout2d(p=0.25, inplace=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=100352, out_features=512, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=512, out_features=2, bias=True)
  )
)

In [ ]:
for key in dataSets:
    print(key)
    if "Model_1" in key:
        model = model_1
    else:
        model = model_2
    
    results = evaluate_model(model.to(DEVICE), dataSets[key], DEVICE)
    print("Accuracy: ", results[0])
    print("F1 Score: ", results[1])
    print("confusion Matrix: \n",results[3])
    print("\n")

Model_1:Train
Accuracy:  0.9283333333333333
F1 Score:  0.9271926854046733
confusion Matrix: 
 [[1416   84]
 [ 131 1369]]


Model_1:Test
Accuracy:  0.87
F1 Score:  0.8617021276595744
confusion Matrix: 
 [[93  7]
 [19 81]]


Model_2:Train
Accuracy:  0.9436666666666667
F1 Score:  0.9452188006482982
confusion Matrix: 
 [[1373  127]
 [  42 1458]]


Model_2:Test
Accuracy:  0.89
F1 Score:  0.8981481481481481
confusion Matrix: 
 [[81 19]
 [ 3 97]]


Model_1:additional_set


In [ ]:
K_ROUND = 50  # Number of rounds to monitor
activation_values = {"Model_1":{},
                     "Model_2":{}}



In [ ]:
def register_hooks(model,model_name):
    hooks = []
    layer_info = {}
    counter = 0

    last_linear_name = None
    for name, layer in model.named_modules():
        if isinstance(layer, nn.Linear):
            last_linear_name = name
    
    global last_layer_name
    last_layer_name = last_linear_name

    for name, layer in model.named_modules():
        #if isinstance(layer, (nn.Conv2d, nn.Linear)):
            # Register hook for the full layer output
            #hooks.append(layer.register_forward_hook(get_activation(name,model_name)))
            
            if isinstance(layer, nn.Linear):
                layer_info[name] = {'type': f'{counter:02}-linear',}
                if name == last_linear_name:
                    layer_info[f"{name}_softmax"] = {'type': f'{counter:02}-softmax'}
                hooks.append(layer.register_forward_hook(get_activation(name,model_name)))
            elif isinstance(layer, nn.Conv2d):
                layer_info[name] = {'type': f'{counter:02}-conv'}
                hooks.append(layer.register_forward_hook(get_activation(name,model_name)))
            # elif isinstance(layer, nn.ReLU):
            #      layer_info[name] = {'type': f'{counter:02}-relu'}
            #      hooks.append(layer.register_forward_hook(get_activation(name,model_name)))
            elif isinstance(layer, nn.MaxPool2d):
                 hooks.append(layer.register_forward_hook(get_activation(name,model_name)))
                 layer_info[name] = {'type': f'{counter:02}-maxpool'}
            # elif isinstance(layer, nn.Dropout):
            #     layer_info[name] = {'type': f'{counter}-dropout'}
            elif isinstance(layer, nn.Flatten):
                 layer_info[name] = {'type': f'{counter}-flatten'}
            # elif isinstance(layer, nn.BatchNorm2d):
            #     layer_info[name] = {'type': f'{counter}-batchnorm'}
            # elif isinstance(layer, nn.AdaptiveAvgPool2d):
            #     layer_info[name] = {'type': f'{counter}-adaptiveavgpool'}
            # elif isinstance(layer, nn.Softmax):
            #     layer_info[name] = {'type': f'{counter}-softmax'}
            # else:
            #     layer_info[name] = {'type': f'{counter}-other'}
            counter += 1

    return hooks, layer_info

def get_activation(name, model_name):
    def hook(module, input, output):
        output_relu = torch.nn.functional.relu(output)
        if name == last_layer_name:
            activation_values[model_name][name] = {
                'output': output_relu.detach(),
                'input': input[0].detach(),  # Capturing input
                'weight': module.weight.detach() if hasattr(module, 'weight') else None
            }
            activation_values[model_name][f"{name}_softmax"] = torch.nn.functional.softmax(output_relu.detach(), dim=1)
        else:
            activation_values[model_name][name] = {
                'output': output_relu.detach(),
                'input': input[0].detach(),  # Capturing input
                'weight': module.weight.detach() if hasattr(module, 'weight') else None
            }
    return hook

In [ ]:
def select_random_neuron_indices(activations, layer_info, n_neurons=1):
    indices = {}
    all_activated_indices = {}
    previous_layer_contributions = {}
    
    # sorted_layers = sorted(layer_info.keys())
    
    for layer_name in layer_info:
        # WILL DELETE
        if layer_name not in activations:
            continue
            
        layer_data = activations[layer_name]
        if isinstance(layer_data, dict): 
            activation = layer_data['output']
            input_data = layer_data['input']
            weights = layer_data['weight']
            
        # original shape
        original_shape = activation.shape
        
        flattened = activation.view(activation.shape[0], -1)
        positive_indices = np.where(flattened.cpu().numpy().squeeze() > 0)[0]
        
        # all activated indices/inputs whatever it is  for this layer
        all_activated_indices[layer_name] = positive_indices
        

        # debug
        if len(positive_indices) == 0:
            continue
            
        # Select random neurons from activated ones
        selected_indices = np.random.choice(
            positive_indices,
            size=min(n_neurons, len(positive_indices)), # good to think about this
            replace=False # no replace
        )
        
        # calculate contributions from previous layer if weights exist eleminate sacma salak seyler
        if weights is not None and input_data is not None:
            contributions = {}
            for selected_idx in selected_indices:
                # For convolutional layers ([B, 32, 224, 224])
                if len(weights.shape) == 4:  
                    # converting flattened index back to feature map coordinates
                    if len(original_shape) == 4:  # batch, channels, height, width
                        batch_size, channels, height, width = original_shape

                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        channel = selected_idx // (height * width)
                        pos_in_channel = selected_idx % (height * width)
                        h_idx = pos_in_channel // width
                        w_idx = pos_in_channel % width
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        
                        # besure channel is within bounds
                        if channel >= weights.shape[0]:
                            continue
                        
                        # Get weights for this output channel
                        channel_weights = weights[channel]
                        
                        # simplified calculate contribution
                        contribution = torch.tensor(0.0)
                        if input_data.shape[2] > h_idx and input_data.shape[3] > w_idx:
                            contribution = input_data[0, :, h_idx, w_idx].sum() * channel_weights.sum()
                    else:
                        # skiped if shape doesnt match expectations
                        #print(f"Skipping layer {layer_name} due to unexpected shape: {original_shape}")
                        continue
                        
                # for linear layers
                else:  
                    # for linear layers, the flattened index needs to be mapped to output dimension
                    output_dim = weights.shape[0]
                    if selected_idx >= output_dim: # its random operation is wron
                        # Skip if index is out of bounds
                        #print(f"Skipping layer {layer_name} due to out of bounds index: {selected_idx}")
                        continue
                        
                    neuron_weights = weights[selected_idx]
                    contribution = (input_data * neuron_weights).sum()
                
                contributions[int(selected_idx)] = {
                    'weights': weights[min(selected_idx, weights.shape[0]-1)].cpu().numpy(),
                    'input_values': input_data.cpu().numpy(),
                    'contribution': contribution.item() if isinstance(contribution, torch.Tensor) else contribution
                }
            
            previous_layer_contributions[layer_name] = contributions
        
        indices[layer_name] = {
            'neuron_idx': selected_indices,
            'type': layer_info[layer_name]['type'],
            'all_activated_indices': positive_indices,
            'contributions': previous_layer_contributions.get(layer_name, None)
        }
    
    return indices, all_activated_indices, previous_layer_contributions

In [ ]:
def compare_model_activations(activation_values_1, activation_values_2, indices1, indices2, layers1, layers2):
    comparison_results = {}
    
    # Get the layer names from Model_1
    for layer_name in layers1:
        if layer_name not in activation_values_1 or layer_name not in activation_values_2:
            continue
            
        data1 = activation_values_1[layer_name]
        data2 = activation_values_2[layer_name]
        
        if not isinstance(data1, dict) or not isinstance(data2, dict):
            continue
            
        input1 = data1['input']
        weights2 = data2['weight']
        #print(weights2)
        #print(layer_name)
        #print(type(weights2))
        bias2 = data2.get('bias', None) #FOR SOFTMAX
        output1 = data1['output']
        
        # Get selected neuron indices
        if layer_name in indices1[0]:
            selected_indices = indices1[0][layer_name]['neuron_idx']
        else:
            #print("Skipping layer", layer_name)
            continue
            
        # results for this layer
        layer_results = {
            'neuron_comparisons': [],
            'mean_difference': 0.0,
            'max_difference': 0.0,
            'min_difference': float('inf')
        }
        
        # Compare selected neurons
        for idx in selected_indices:
            if weights2 is not None:
                # CONV
                if len(weights2.shape) == 4:  # Convolutional layer
                    
                    batch_size, channels, height, width = output1.shape
                    channel = idx // (height*width)
                    pos_in_channel = idx%(height * width)
                    h_idx = pos_in_channel//width
                    w_idx = pos_in_channel%width
                    
                    if channel >= weights2.shape[0]:
                        #print(f"Skipping layer {layer_name} due to out of bounds channel: {channel}")
                        continue
                        
                    # Get actual activation from Model_1
                    actual_activation = output1[0, channel, h_idx, w_idx].item()
                    
                    kernel_size = weights2.shape[2]
                    padding = kernel_size // 2 
                    
                                            # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                                                # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                                                # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                                                # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                                                # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                                                # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                    # Extract input patch
                    in_h_start = max(0, h_idx - padding)
                    in_w_start = max(0, w_idx - padding)
                    in_h_end = min(input1.shape[2],h_idx+kernel_size-padding)
                    in_w_end = min(input1.shape[3],w_idx+kernel_size-padding)
                                            # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                                                # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                                                # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                                                # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                                                # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                                                # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                        # DONT TOUCH THIS PART
                    input_patch = input1[0, :, in_h_start:in_h_end, in_w_start:in_w_end]
                    weight_patch = weights2[channel]
                    
                    # Adjust weight patch size if needed
                    if input_patch.shape != weight_patch.shape[1:]:
                        bad_model_activation = torch.tensor(0.0)
                    else:
                        bad_model_activation = (input_patch * weight_patch).sum()
                    
                    # Add bias if exist
                    if bias2 is not None:
                        bad_model_activation += bias2[channel]
                    
                    # apply ReLU for any case
                    if actual_activation > 0:
                        bad_model_activation = max(0, bad_model_activation)
                    
                else:  # Linear layer
                    #if index is out of bounds
                    if idx >= weights2.shape[0]:
                        continue
                        
                    # Get actual activation from Model_1 (flattened)
                    flattened_output = output1.view(output1.shape[0], -1)
                    actual_activation = flattened_output[0, idx].item()
                    
                    # Calculate expected activation using Model_2 weights
                    neuron_weights = weights2[idx]
                    bad_model_activation = (input1.flatten(start_dim=1) * neuron_weights).sum()
                    
                    # Add bias if available
                    if bias2 is not None:
                        bad_model_activation += bias2[idx]
                    
                    if actual_activation > 0:
                        bad_model_activation = max(0, bad_model_activation)
                
                #converting to scalar if it is a tensor
                if isinstance(bad_model_activation, torch.Tensor):
                    bad_model_activation = bad_model_activation.item()
                    
                #calculate difference
                difference = abs(actual_activation - bad_model_activation)
                
                #comparison for this neuron
                neuron_comparison = {
                    'neuron_idx': int(idx),
                    'actual_activation': actual_activation,
                    'bad_model_activation': bad_model_activation,
                    'difference': difference,
                    'percen_diff': (difference / (abs(actual_activation) + 1e-10)) * 100
                }
                
                layer_results['neuron_comparisons'].append(neuron_comparison)
                
                #statistics
                layer_results['mean_difference'] += difference
                layer_results['max_difference'] = max(layer_results['max_difference'], difference)
                layer_results['min_difference'] = min(layer_results['min_difference'], difference)
            
            #average difference
            if len(layer_results['neuron_comparisons']) > 0:
                layer_results['mean_difference'] /= len(layer_results['neuron_comparisons'])
            else:
                layer_results['mean_difference'] = float('nan')
                layer_results['min_difference'] = float('nan')
            
            comparison_results[layer_name] = layer_results
    return comparison_results
    

In [ ]:
def save_comparison_results_to_csv(comparison_results, input_id, round_id, filename="model_comparison_results.csv"):
    rows = []
    for layer_name, layer_data in comparison_results.items():
        mean_diff = layer_data['mean_difference']
        max_diff = layer_data['max_difference']
        min_diff = layer_data['min_difference']
    
        if np.isnan(mean_diff):
            mean_diff = "N/A"
        if np.isnan(min_diff):
            min_diff = "N/A"
    
        if layer_data['neuron_comparisons']:
            for neuron_data in layer_data['neuron_comparisons']:
                row = {
                    'input_id': input_id,
                    'round_id': round_id,
                    'layer_name': layer_name,
                    'neuron_idx': neuron_data['neuron_idx'],
                    'actual_activation': neuron_data['actual_activation'],
                    'bad_model_activation': neuron_data['bad_model_activation'],
                    'difference': neuron_data['difference'],
                    'percent_difference': neuron_data['percen_diff'],
                    'layer_mean_difference': mean_diff,
                    'layer_max_difference': max_diff,
                    'layer_min_difference': min_diff
                }
                rows.append(row)
        else:
            row = {
                'input_id': input_id,
                'round_id': round_id,
                'layer_name': layer_name,
                'neuron_idx': "N/A",
                'actual_activation': "N/A",
                'bad_model_activation': "N/A",
                'difference': "N/A",
                'percent_difference': "N/A",
                'layer_mean_difference': mean_diff,
                'layer_max_difference': max_diff,
                'layer_min_difference': min_diff
            }
            rows.append(row)
    
    df = pd.DataFrame(rows)
    file_exists = os.path.isfile(filename)
    
    if file_exists:
        df.to_csv(filename, mode='a', header=False, index=False)
        #print(f"Results appended to {filename}")
    else:
        df.to_csv(filename, index=False)
        #print(f"Results saved to new file {filename}")
    
    return df

In [ ]:
classes = {
    0: 'cat',
    1: 'dog'
}

In [ ]:
the_class = 1 # 0 cat or squirrel, 1 dog

In [ ]:
# activation_values = {"Model_1":{},
#                     "Model_2":{}}

In [ ]:
# images = [(img, idx) for idx, (img, label) in enumerate(dataSets["Model_1:Test"].dataset) 
#             if label == 0][:15]
# image = images[0][0]

In [ ]:
# hooks1, layers1 = register_hooks(model_1, "Model_1")
# hooks2, layers2 = register_hooks(model_2, "Model_2")

# with torch.no_grad():
#     pred1 = model_1(image.unsqueeze(0).to(DEVICE))
#     pred2 = model_2(image.unsqueeze(0).to(DEVICE))

# indices1 = select_random_neuron_indices(activation_values["Model_1"], layers1)
# indices2 = select_random_neuron_indices(activation_values["Model_2"], layers2)

# comparison_results = compare_model_activations(activation_values["Model_1"], 
#                                                activation_values["Model_2"], 
#                                                indices1, 
#                                                indices2, 
#                                                layers1, 
#                                                layers2)

# for layer_name, results in comparison_results.items():
#     print(f"\nLayer: {layer_name}")
#     print(f"Mean difference: {results['mean_difference']:.6f}")
#     print(f"Max difference: {results['max_difference']:.6f}")
#     print(f"Min difference: {results['min_difference']:.6f}")

#     print("\nSample neuron comparisons:")
#     for i, comp in enumerate(results['neuron_comparisons'][:5]): 
#         print(f"  Neuron {comp['neuron_idx']}: Actual={comp['actual_activation']:.6f}, "
#               f"Prime activation={comp['bad_model_activation']:.6f}, "
#               f"Diff={comp['difference']:.6f} ({comp['percen_diff']:.2f}%)")
    
#     good_matches = sum(1 for comp in results['neuron_comparisons'] 
#                       if comp['percen_diff'] < 5)  # Consider <5% difference
#     match_percentage = (good_matches / len(results['neuron_comparisons'])) * 100 if results['neuron_comparisons'] else 0
    
#     print(f"Match percentage (within 5% difference): {match_percentage:.2f}%")

# save_comparison_results_to_csv(comparison_results, 0, 0, filename="model_comparison_results.csv")
# for hook in hooks1 + hooks2:
#     hook.remove()

tensor([[[[ 1.0438e-01,  1.9694e-01,  4.6683e-02],
          [ 1.4248e-01, -7.4677e-02,  2.9127e-02],
          [-1.5066e-01, -7.6323e-02, -1.4803e-01]],

         [[ 5.3139e-02,  1.8019e-02,  1.9684e-02],
          [ 7.0723e-02, -2.1554e-01, -1.9714e-01],
          [-1.7370e-01,  3.0297e-02, -2.4181e-01]],

         [[ 1.3528e-01,  8.8570e-02,  1.8613e-01],
          [-9.1233e-02, -1.1921e-01, -1.2085e-01],
          [-8.4497e-02, -2.7555e-02, -6.5062e-03]]],


        [[[-2.5765e-01,  8.4153e-02, -4.2445e-02],
          [ 6.4696e-02,  6.7702e-02, -2.8961e-03],
          [-6.4010e-02,  1.6071e-01,  9.5482e-02]],

         [[ 3.4415e-02, -5.0169e-02,  1.4346e-01],
          [ 8.7364e-02, -1.1939e-01, -2.0954e-01],
          [ 1.8741e-01,  1.5890e-01, -1.8179e-01]],

         [[-2.5901e-01, -1.4860e-01,  4.0433e-02],
          [-2.2548e-01,  8.2531e-03,  1.7558e-02],
          [-3.1579e-02,  1.1798e-02, -2.7838e-01]]],


        [[[-1.9100e-01,  1.9106e-01, -1.3767e-01],
          [ 1.4

In [ ]:
with torch.no_grad():
    images = [(img, idx) for idx, (img, label) in enumerate(dataSets["Model_1:additional_set"].dataset) 
                if label == the_class and model_1(img.unsqueeze(0).to(DEVICE))
                                          .argmax().item() == the_class]

all_dfs = []

for image, pic_index in images:
    for i in range(K_ROUND):
        activation_values = {"Model_1":{},
                    "Model_2":{}}
        hooks1, layers1 = register_hooks(model_1, "Model_1")
        hooks2, layers2 = register_hooks(model_2, "Model_2")

        with torch.no_grad():
            pred1 = model_1(image.unsqueeze(0).to(DEVICE))
            pred2 = model_2(image.unsqueeze(0).to(DEVICE))

        indices1 = select_random_neuron_indices(activation_values["Model_1"], layers1)
        indices2 = select_random_neuron_indices(activation_values["Model_2"], layers2)

        comparison_results = compare_model_activations(activation_values["Model_1"], 
                                                    activation_values["Model_2"], 
                                                    indices1, 
                                                    indices2, 
                                                    layers1, 
                                                    layers2)

        # for layer_name, results in comparison_results.items():
        #     print(f"\nLayer: {layer_name}")
        #     print(f"Mean difference: {results['mean_difference']:.6f}")
        #     print(f"Max difference: {results['max_difference']:.6f}")
        #     print(f"Min difference: {results['min_difference']:.6f}")

        #     print("\nSample neuron comparisons:")
        #     for i, comp in enumerate(results['neuron_comparisons'][:5]): 
        #         print(f"  Neuron {comp['neuron_idx']}: Actual={comp['actual_activation']:.6f}, "
        #             f"Prime activation={comp['bad_model_activation']:.6f}, "
        #             f"Diff={comp['difference']:.6f} ({comp['percen_diff']:.2f}%)")
            
        #     good_matches = sum(1 for comp in results['neuron_comparisons'] 
        #                     if comp['percen_diff'] < 5)  # Consider <5% difference
        #     match_percentage = (good_matches / len(results['neuron_comparisons'])) * 100 if results['neuron_comparisons'] else 0
            
        #     print(f"Match percentage (within 5% difference): {match_percentage:.2f}%")
        print('\r ',"pic_index: ", pic_index, "round: ",i, end="")
        save_comparison_results_to_csv(comparison_results, pic_index, i, filename=f"model_comparison_results_on_{classes[the_class]}.csv")
        for hook in hooks1 + hooks2:
            hook.remove()